In [8]:
import pickle
from MINE.mi import MineClassif, MineOpt
from MINE.augmentation import *
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Model
from tensorflow.keras import models as keras_models
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.rcParams['figure.figsize'] = 20, 15
import os
import pickle
import pandas as pd

In [34]:
def construct_model(input_x_shape, input_z_shape, activation):
    inp_x = Input((input_x_shape, ))
    inp_z = Input((input_z_shape, ))

    x = layers.Concatenate()([inp_x, inp_z])
    x = layers.Dense(64, activation='tanh')(x)
    x = layers.Dense(1, activation=activation)(x)
    
    model = Model(inputs=[inp_x, inp_z], outputs=x)
    return model

In [35]:
def create_uni(n, d):
    tmp = np.random.choice(range(d), n)
    tmp2 = np.zeros((tmp.size, tmp.max() + 1))
    tmp2[np.arange(tmp.size), tmp] = 1
    return tmp2

In [38]:
def fit_model_opt(dataset, approx, L, lam, C, x_ind, y_ind):
    model = construct_model(len(x_ind), len(y_ind))
    mine = MineOpt(model, approximation=approx, L=L, lam=lam, C=C)
    mine.compile(optimizer='adam')
    es = tf.keras.callbacks.EarlyStopping(patience=20, monitor='mi', mode='max')
    history = mine.fit([dataset[:, x_ind], dataset[:, y_ind]], None, batch_size=256, epochs=1000, callbacks=[es], verbose=0)
    return mine

def fit_model_classif(dataset, x_ind, y_ind):
    model = construct_model(len(x_ind), len(y_ind))
    mine = Mine(model)
    mine.compile(loss='binary_crossentropy', optimizer='adam')
    es = tf.keras.callbacks.EarlyStopping(patience=20, monitor='loss', mode='min')
    history = mine.fit([dataset[:, x_ind], dataset[:, y_ind]], None, batch_size=256, epochs=1000, callbacks=[es], verbose=0)
    return mine

In [24]:
np.random.seed(77)

uni = {100: {}, 1000: {}, 10_000: {}}
uni[100]['orig'] = [create_uni(100, 16) for i in range(30)]
uni[1000]['orig'] = [create_uni(1000, 16) for i in range(30)]
uni[10_000]['orig'] = [create_uni(10_000, 16) for i in range(30)]


norm_not_corr = {100: {}, 1000: {}, 10_000: {}}
mean = np.array([0, 1])
cov = np.array([[1, 0], [0, 2]])
norm_not_corr[10_000]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=10_000) for i in range(30)]
norm_not_corr[1_000]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=1_000) for i in range(30)]
norm_not_corr[100]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=100) for i in range(30)]

norm_corr = {}
mean = np.array([0, 1])
cov = np.array([[1, 0.75], [0.75, 2]])
norm_corr = {100: {}, 1000: {}, 10_000: {}}
norm_corr[10_000]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=10_000) for i in range(30)]
norm_corr[1_000]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=1_000) for i in range(30)]
norm_corr[100]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=100) for i in range(30)]

norm_hd = {}
mean = np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])
A = np.random.uniform(-1, 1, 81).reshape(9, 9)
cov = np.matmul(A, A.T)
norm_hd = {100: {}, 1000: {}, 10_000: {}}
norm_hd[10_000]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=10_000) for i in range(30)]
norm_hd[1_000]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=1_000) for i in range(30)]
norm_hd[100]['orig'] = [np.random.multivariate_normal(mean=mean, cov=cov, size=100) for i in range(30)]

datasets = {'uni': uni, 'norm_not_corr': norm_not_corr, 'norm_corr': norm_corr, 'norm_hd': norm_hd}

aug = Augmentation()

for dataset_type_name, dataset_type in datasets.items():
    for dataset_size_name, dataset_size in dataset_type.items():
        dataset_size['aug'] = [aug.transform(dataset_size['orig'][i], n=10, m=1) for i in range(30)]

In [ ]:
models = {}
for dataset_type_name, dataset_type in tqdm(datasets.items()):
    models[dataset_type_name] = {}
    for dataset_size_name, dataset_size in tqdm(dataset_type.items()):
        models[dataset_type_name][dataset_size_name] = {'orig':
                                     {'DV': {'l0': [],
                                             'l0.1': []
                                            },
                                      'FD': {'l0': [],
                                             'l0.1': []
                                            },
                                     },
                                     'aug': {'DV': {'l0': [],
                                             'l0.1': []
                                            },
                                              'FD': {'l0': [],
                                                     'l0.1': []
                                                    }
                                     }
                                    }
        for i in range(30):
            mine = fit_model(dataset_size['orig'][i], 'Donsker_Varadhan', L=None)
            models[dataset_type_name][dataset_size_name]['orig']['DV']['l0'].append(mine)
            
            mine = fit_model(dataset_size['orig'], 'Donsker_Varadhan', L=2, lam=0.1, C=0)
            models[dataset_type_name][dataset_size_name]['orig']['DV']['l0.1'].append(mine)
            
            mine = fit_model(dataset_size['orig'], 'f_divergence', L=None)
            models[dataset_type_name][dataset_size_name]['orig']['FD']['l0'].append(mine)
            
            mine = fit_model(dataset_size['orig'], 'f_divergence', L=2, lam=0.1, C=1)
            models[dataset_type_name][dataset_size_name]['orig']['FD']['l0.1'].append(mine)

            models[dataset_type_name][dataset_size_name]['aug']['DV']['l0'].append([])
            models[dataset_type_name][dataset_size_name]['aug']['DV']['l0.1'].append([])
            models[dataset_type_name][dataset_size_name]['aug']['FD']['l0'].append([])
            models[dataset_type_name][dataset_size_name]['aug']['FD']['l0.1'].append([])
            for j in range(10):
                mine = fit_model(dataset_size['aug'][i][j], 'Donsker_Varadhan', None)
                models[dataset_type_name][dataset_size_name]['aug']['DV']['l0'][-1].append(mine)
                
                mine = fit_model(dataset_size['aug'][i][j], 'Donsker_Varadhan', L=2, lam=0.1, C=0)
                models[dataset_type_name][dataset_size_name]['aug']['DV']['l0.1'][-1].append(mine)
                
                mine = fit_model(dataset_size['aug'][i][j], 'f_divergence', L=None)
                models[dataset_type_name][dataset_size_name]['aug']['FD']['l0'][-1].append(mine)
                
                mine = fit_model(dataset_size['aug'][i][j], 'f_divergence', L=2, lam=0.1, C=1)
                models[dataset_type_name][dataset_size_name]['aug']['FD']['l0.1'][-1].append(mine)

In [ ]:
def initial_experiments(n, seed):
    seed = int(seed)
    tf.keras.utils.set_random_seed(seed)

    datasets = prepare_datasets(n)

    result = {}

    for dataset_name, dataset in datasets.items():
        result[dataset_name] = {n: {'orig':
                                     {'DV': {'l0': {seed: None},
                                             'l0.1': {seed: None}
                                            },
                                      'FD': {'l0': {seed: None},
                                             'l0.1': {seed: None}
                                            },
                                     },
                                     'aug': {'DV': {'l0': {seed: []},
                                             'l0.1': {seed: []}
                                            },
                                              'FD': {'l0': {seed: []},
                                                     'l0.1': {seed: []}
                                                    }
                                     }
                                    }
        print(":)")
        if 'aug' not in dataset_name:
            m = fit_model(dataset, 'Donsker_Varadhan', L=None)
            result[dataset_name][n]['orig']['DV']['l0'][seed] = m

            m = fit_model(dataset, 'Donsker_Varadhan', L=2, lam=0.1, C=0)
            result[dataset_name][n]['orig']['DV']['l0.1'][seed] = m

            m = fit_model(dataset, 'f_divergence', L=None)
            result[dataset_name][n]['orig']['f_divergence']['l0'][seed] = m

            m = fit_model(dataset, 'f_divergence', L=2, lam=0.1, C=1)
            result[dataset_name][n]['orig']['f_divergence']['l0.1'][seed] = m
        else:
            for i in range(10):
                m = fit_model(dataset, 'Donsker_Varadhan', L=None)
                result[dataset_name][n]['aug']['DV']['l0'][seed].append(m)

                m = fit_model(dataset, 'Donsker_Varadhan', L=2, lam=0.1, C=0)
                result[dataset_name][n]['aug']['DV']['l0.1'][seed].append(m)

                m = fit_model(dataset, 'f_divergence', L=None)
                result[dataset_name][n]['aug']['f_divergence']['l0'][seed].append(m)

                m = fit_model(dataset, 'f_divergence', L=2, lam=0.1, C=1)
                result[dataset_name][n]['aug']['f_divergence']['l0.1'][seed].append(m)

    dt = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')

    with open(f'results/{n}_{seed}_{dt}.pkl', 'wb') as fd:
        pickle.dump(results, fd)